In [ ]:
# importar bibliotecas
from sklearn.model_selection import train_test_split
import json
import ast
from spacy.ml.models import load_kb
import random  
from spacy.util import minibatch, compounding

In [ ]:
# activar gpu
activated = spacy.prefer_gpu()
spacy.require_gpu()

#cargar modelo
nlp = spacy.load("en_core_web_sm")

In [ ]:
# cargar dataset que sirve de entrenamiento
dataframeNEL = pd.read_csv("train_dataset.csv",encoding="ISO-8859-1")

In [ ]:
# Nombrar columnas
dataframeNEL = dataframeNEL.set_axis(['Reports', 'Labels'], axis=1)

In [ ]:
# eliminar duplicados
dataframeNEL = dataframeNEL.drop_duplicates('Reports')
len(dataframeNEL)

In [ ]:
# obtener primeros 3000 registros
dataframeNEL = dataframeNEL.head(3000)

In [ ]:
# crear diccionario para enviar al proceso de entrenamiento
dataset = []
for index, row in dataframeNEL.iterrows():
    dataset.append((row["Reports"],ast.literal_eval(row["Labels"])))
    

In [ ]:
# crear objeto para hacer division entre conjunto de entrenamiento y pruebas
xs = []
ys = []            

for index, (text, annot) in enumerate(dataset):

  y = []

  subLinks = []
  for span, links_dict in annot["links"].items():
    
          
    for link, value in links_dict.items():
      
      link = int(link)
      if link in name_dict.keys() and value:
        subLinks.append(link)
    
  y.append(subLinks)
  if len(y) != 0:
    xs.append(dataset[index])  
    ys.append(y[0])   


train_dataset, test_dataset, y_train, y_test = train_test_split(xs,ys,test_size=0.33, random_state=42)


print (f"Number of sentences. Total: {len(dataset)}. Number of training sentences: {len(train_dataset)}. Number of test sentences: {len(test_dataset)}")
print (*[f"Test sentence: text -> '{text}'\t annotation -> {annot}" for text, annot in test_dataset], sep = "\n")

In [ ]:
from spacy.training import Example

In [ ]:
# crear objeto example que se requier enviar a entity linker
TRAIN_EXAMPLES = []
if "sentencizer" not in nlp.pipe_names:
  nlp.add_pipe("sentencizer")

sentencizer = nlp.get_pipe("sentencizer")
for text, annotation in train_dataset:
  print(text)
  example = Example.from_dict(nlp.make_doc(text), annotation)
  example.reference = sentencizer(example.reference)
  TRAIN_EXAMPLES.append(example)
print ("'train_dataset' has been transformed to 'train_examples', a dataset with spaCy 'example' objects")

In [ ]:
# agregar entity linker a pipeline
entity_linker = nlp.add_pipe("entity_linker", config={"incl_prior": False}, last=True)

In [ ]:
# inicializar entity linker con objeto Example
entity_linker.initialize(get_examples=lambda: TRAIN_EXAMPLES, kb_loader=load_kb("nel_kb_v3"))

In [ ]:
#entrenamiento de entity linker, en caso de querer entrenar entidades, colocar ner en la lista de enable
with nlp.select_pipes(enable=["entity_linker"]): 
    optimizer = nlp.resume_training()
    for itn in range(1000):   
        random.shuffle(TRAIN_EXAMPLES)
        batches = minibatch(TRAIN_EXAMPLES, size=compounding(4.0, 32.0, 1.001))  # increasing batch sizes
        losses = {}
        for batch in batches:
            nlp.update(
                batch,   
                drop=0.2,      
                losses=losses,
                sgd=optimizer,
            )
        if itn % 50 == 0:
            print(itn, "Losses", losses)   # print the training loss
print(itn, "Losses", losses)

In [ ]:
nlp.to_disk("NEL_Model_2")

In [ ]:
# Ejemplo de predicción
text = "very sick; horrible; has allergy to environmental items like dust/grass; sinuses were irritating; they were \"hoping tomorrow [they would be] back to [their] real normal; headache; stomach ache; fever; aches; diarrhea; booster; whole body hurts; breaking out on their face/looked like redness has spread slightly beyond bite area (no threat); had rash developing in cracks of lips that was told to me yesterday; This is a spontaneous report received from a contactable reporter(s) (Consumer or other non HCP).  A 54 year-old female patient (not pregnant) received bnt162b2 (BNT162B2), administration date 10Dec2021 (Batch/Lot number: unknown) at the age of 54 years as dose 3 (booster), single for covid-19 immunisation. Relevant medical history included: \"history anxiety/depression\" (unspecified if ongoing); \"history anxiety/depression\" (unspecified if ongoing); \"high blood pressure\" (unspecified if ongoing); \"cosmetic related surgery earlier this year\", start date: 2021 (unspecified if ongoing), notes: other medical history: cosmetic related surgery earlier this year; \"scoliosis with implanted spinal rod and associated back/neck pain\" (unspecified if ongoing); \"scoliosis with implanted spinal rod and associated back/neck pain\" (unspecified if ongoing); \"scoliosis with implanted spinal rod and associated back/neck pain\" (unspecified if ongoing); \"Reaction : Allergy\" (unspecified if ongoing); \"Covid\" (unspecified if ongoing), notes: did have Covid at some point before being vaccinated. The patient took concomitant medications. Vaccination history included: Bnt162b2 (Dose Number: 2, Batch/Lot No: Unknown. Not available/provided to reporter at the time of report completion, prev dose administration date: 10Jun2021), administration date: 10Jun2021, when the patient was 54 years old, for COVID-19 immunisation; Bnt162b2 (Dose Number: 1, Batch/Lot No: Unknown., prev dose administration date: 20May2021), administration date: 20May2021, when the patient was 54 years old, for COVID-19 immunisation. The following information was reported: IMMUNISATION (non-serious) with onset 10Dec2021, outcome \"unknown\", described as \"booster\"; PAIN (non-serious) with onset 10Dec2021, outcome \"not recovered\", described as \"whole body hurts\"; HEADACHE (non-serious) with onset 12Dec2021, outcome \"not recovered\", described as \"headache\"; PYREXIA (non-serious) with onset 10Dec2021, outcome \"not recovered\", described as \"fever\"; PAIN (non-serious) with onset 10Dec2021, outcome \"not recovered\", described as \"aches\"; DIARRHOEA (non-serious) with onset 10Dec2021, outcome \"not recovered\", described as \"diarrhea\"; ABDOMINAL PAIN UPPER (non-serious) with onset 12Dec2021, outcome \"unknown\", described as \"stomach ache\"; FEELING ABNORMAL (non-serious) with onset 12Dec2021 20:21, outcome \"unknown\", described as \"they were \"hoping tomorrow [they would be] back to [their] real normal\"; ERYTHEMA (non-serious) with onset Dec2021, outcome \"unknown\", described as \"breaking out on their face/looked like redness has spread slightly beyond bite area (no threat)\"; ILLNESS (non-serious), outcome \"unknown\", described as \"very sick\"; MALAISE (non-serious), outcome \"unknown\", described as \"horrible\"; HYPERSENSITIVITY (non-serious), outcome \"unknown\", described as \"has allergy to environmental items like dust/grass\"; SINUS DISORDER (non-serious) with onset 24Dec2021, outcome \"unknown\", described as \"sinuses were irritating\"; RASH (non-serious) with onset Dec2021, outcome \"unknown\", described as \"had rash developing in cracks of lips that was told to me yesterday\". Relevant laboratory tests and procedures are available in the appropriate section.  Follow-up (11Dec2021): This is a spontaneous follow-up report from same contactable consumer. This consumer reported for a 54-year-old female patient that:  Updated information included : Historical vaccine, Relevant medical history, New event of \"stomach ache\".  Follow-up (12Dec2021): This is a spontaneous follow-up report from same contactable consumer. This consumer reported for a patient that:  Updated information included : New event of \"they were \"hoping tomorrow [they would be] back to [their] real normal\".  Follow-up (17Dec2021): This is a spontaneous follow-up report from same contactable consumer. This consumer reported in response to non-HCP letter sent via Email follow-up activity that included:  Updated information included : Reporter's Phone number  Follow-up (20Dec2021): This is a spontaneous follow-up report from same contactable consumer. This consumer reported in response to non-HCP letter sent via Email follow-up activity that included:  Updated information included: Historical vaccine start and stop date.  Follow-up (20Dec2021): This is a spontaneous follow-up report from same contactable consumer. This consumer reported in response to non-HCP letter sent via Email follow-up activity that included:  Additional information : Sorry third email but one more detail - there are many other reports submitted previously that this one can be linked to. All with same patient initials and DOB thank you,  Follow-up (20Dec2021): This is a spontaneous follow-up report from same contactable consumer. This consumer reported for a patient that:  Updated information included: Relevant Medical History.  Follow-up (20Dec2021): This is a spontaneous follow-up report from same contactable consumer. This consumer reported for a patient that:  Updated information included: New event of \"very sick, horrible\".  Follow-up (25Dec2021): This is a spontaneous follow-up report from same contactable consumer. This consumer reported for a patient that:  Follow-up (24Dec2021): This is a spontaneous follow-up report from same contactable consumer. This consumer reported in response to non-HCP letter sent via Email follow-up activity that included:  Updated information included : New event of  \"has allergy to environmental items like dust/grass\"  Follow-up (24Dec2021): This is a spontaneous follow-up report from same contactable consumer. This consumer reported in response to non-HCP letter sent via Email follow-up activity that included:  Follow-up (25Dec2021): This is a spontaneous follow-up report from same contactable consumer. This consumer reported for a patient that:  Updated information included : Lab test, New event of \"sinuses were irritating\".  Follow-up (24Dec2021): This is a spontaneous follow-up report from same contactable consumer. This consumer reported in response to non-HCP letter sent via Email follow-up activity that included:  Follow-up (26Dec2021): This is a spontaneous follow-up report from same contactable consumer. This consumer reported for a patient that:  Updated information included : Lab test, New event of \"had rash developing in cracks of lips that was told to me yesterday\".  The lot number for bnt162b2 was not provided and will be requested during follow up."
doc = nlpTest(text)
for ent in doc.ents:
  print(f"Mention: {ent.text}. Entity type: {ent.label_}. Entity ID: {ent.kb_id_}")

In [ ]:
# realizar predicciones para el conjunto de pruevas

field_names=["text","Gold annotation","ent_text","ent_labels","annotations"]
    
with open('testDataset.csv', 'a', newline='', encoding="utf-8") as f_object:
      

    dictwriter_object = DictWriter(f_object, fieldnames=field_names)
        
    for text, true_annot in test_dataset:
            
        entitiesText = []
        entitiesLabels = []
        annoTest = []
            
             
        doc = nlp(text)  # to make this more efficient, you can use nlp.pipe() just once for all the texts
        for ent in doc.ents:
            entitiesText.append(ent.text)
            entitiesLabels.append(ent.label_)
            annoTest.append(ent.kb_id_)
        dictwriter_object.writerow({"text":text,"Gold annotation":true_annot,"ent_text":entitiesText,"ent_labels":entitiesLabels, "annotations":annoTest}) 

          

#Close the file object
f_object.close()

In [ ]:
# crear arreglo docs con predicciones en el conjunto de pruebas
docs = []
for text, true_annot in test_dataset:
        doc = nlp(text)
        docs.append(doc)



In [ ]:
# convertir documento para poder hacer etiquetado
from spacy.tokens import DocBin
doc_bin = DocBin(docs=docs)
doc_bin.to_disk("testData.spacy")

In [ ]:
# Muestra de etiqueda con modelo
from spacy.tokens import DocBin
import random
doc_bin = DocBin().from_disk("testData.spacy")

docs = list(doc_bin.get_docs(nlpTest.vocab))


for i, doc in enumerate(random.sample(docs,5)):
  print(f"\n## Sentence {i+1}:")
  spacy.displacy.render(doc, style="ent", jupyter=True) 


In [ ]:
# crear objeto con datos de conjunto de prueba
dataset = []
for index, row in testData.iterrows():
    dataset.append((row["Reports"],ast.literal_eval(row["Gold_annotation"])))
    

In [ ]:
xs = []
ys = []        

# crear objeto para poder evaluar calidad en el conjunto de prueba
for index, (text, annot) in enumerate(dataset):

  y = []

  subLinks = []
  for span, links_dict in annot["links"].items():
    
          
    for link, value in links_dict.items():
      
      link = int(link)
      if link in name_dict.keys() and value:
        subLinks.append(link)
    
  y.append(subLinks)
  if len(y) != 0:
    xs.append(dataset[index])  
    ys.append(y[0])   

In [ ]:
# evaluación de calidad en conjunto de prueba
examples = []
data = xs
for text, annots in data:
    doc = nlpTest.make_doc(text)
    examples.append(Example.from_dict(doc, annots))
print(nlpTest.evaluate(examples)) # This will provide overall and per entity metrics

In [ ]:
# Para elaborar este código se utiliza como ejemplo el trabajo realizado por Nadjet Bouayad-Agha y demás colaboradores en https://drive.google.com/drive/folders/1D30X0jUpaGLwsQckNybs6D7-00Nj4SVn